# All-Sky Plot Utilities

This notebook provides a user's guide to functions in `desiutil.plots` for producing all-sky plots. Refer to the docstring for each function for details on its parameters and usage. Operations involving healpix maps require that the `healpy` package is installed (not available via conda, so use `pip` instead).

In [ ]:
%matplotlib inline
import warnings
from types import MethodType
import numpy as np
import numpy.ma
import matplotlib.pyplot as plt
import matplotlib.colors
from matplotlib.projections import get_projection_names
from astropy.coordinates import SkyCoord, HeliocentricTrueEcliptic, ICRS, SkyOffsetFrame, Longitude
import astropy.units as u

In [ ]:
from desiutil.plots import prepare_data, init_sky

In [ ]:
def plot_grid_map(data, ra_edges, dec_edges, cmap='viridis', colorbar=True,
                  label=None, ax=None, center=None):
    """Plot an array of 2D values using an all-sky projection.

    Pass the data array through :func:`prepare_data` to select a subset to plot
    and clip the color map to specified values or percentiles.

    This function is similar to :func:`plot_healpix_map` but is generally faster
    and has better handling of RA wrap around artifacts.

    Parameters
    ----------
    data : array or masked array
        2D array of data associated with each grid cell, with shape
        ``(n_ra, n_dec)``. Use the output of :func:`prepare_data` as a convenient
        way to specify data cuts and color map clipping.
    ra_edges : array
        1D array of ``n_ra+1`` RA grid edge values in degrees, which must span the
        full circle, *i.e.*, ``ra_edges[0] == ra_edges[-1] - 360``. The RA grid
        does not need to match the edges of the projection, in which
        case any wrap-around cells will be duplicated on both edges.
    dec_edges : array
        1D array of ``n_dec+1`` Dec grid edge values in degrees.  Values are not
        required to span the full range ``[-90, +90]``.
    cmap : colormap name or object, optional
        Matplotlib colormap to use for mapping data values to colors.
    colorbar : bool, optional
        Draw a colorbar below the map when ``True``.
    label : str, optional
        Label to display under the colorbar.  Ignored unless colorbar is ``True``.
    ax : :class:`~matplotlib.axes.Axes`, optional
        Axes to use for drawing this map, or create default axes using
        :func:`init_sky` when ``None``.

    Returns
    -------
    :class:`~matplotlib.axes.Axes`
        The axis object used for the plot.
    """
    data = prepare_data(data)
    if len(data.shape) != 2:
        raise ValueError('Expected 2D data array.')
    n_dec, n_ra = data.shape
    #
    # Normalize the data using its vmin, vmax attributes, if present.
    # Need to do this before hstack-ing below, which drops vmin, vmax.
    #
    try:
        norm = matplotlib.colors.Normalize(vmin=data.vmin, vmax=data.vmax)
    except AttributeError:
        norm = None
    #
    # Silently flatten, sort, and remove duplicates from the edges arrays.
    #
    ra_edges = np.unique(ra_edges)
    dec_edges = np.unique(dec_edges)
    if len(ra_edges) != n_ra + 1:
        raise ValueError('Invalid ra_edges.')
    if len(dec_edges) != n_dec + 1:
        raise ValueError('Invalid dec_edges.')
    if ra_edges[0] != ra_edges[-1] - 360:
        raise ValueError('Invalid ra_edges, do not span 360 degrees.')
    #
    # Create axes.
    #
    if ax is None:
        ax = init_sky()
    #
    # Find the projection edge.
    #
    proj_edge = (ax._ra_center - Longitude(180.0*u.deg)).degree
    while proj_edge < 360:
        proj_edge += 360
    #
    # Shift RA gridlines so they overlap the map's left-edge RA.
    #
    while ra_edges[0] > lonmin:
        ra_edges -= 360
    while ra_edges[0] <= lonmin - 360:
        ra_edges += 360
    #
    # Find the first RA gridline that fits within the map's left edge.
    #
    first = np.where(ra_edges >= lonmin)[0][0]

    if first > 0:
        # Wrap the data beyond the left edge around to the right edge.
        # Remember to use numpy.ma.hstack for the data to preserve the mask.
        if ra_edges[first] > lonmin:
            # Split a wrap-around column into separate left and right columns.
            ra_edges = np.hstack(([lonmin], ra_edges[first:],
                                  ra_edges[:first] + 360, [lonmax]))
            data = numpy.ma.hstack(
                (data[:, first - 1:first], data[:, first:],
                 data[:, :first], data[:, first:first + 1]))
        else:
            ra_edges = np.hstack((ra_edges[first:], ra_edges[:first + 1] + 360))
            data = numpy.ma.hstack((data[:, first:], data[:, :first + 1]))

    # Build a 2D array of grid line intersections.
    grid_ra, grid_dec = np.meshgrid(ra_edges, dec_edges)

    # print(grid_ra, grid_dec)
    mesh = ax.pcolormesh(
        -1.0*np.radians(grid_ra), np.radians(grid_dec), data, cmap=cmap, norm=norm,
        edgecolor='none', lw=0)

    if colorbar:
        bar = plt.colorbar(
            mesh, ax=ax, orientation='horizontal',
            spacing='proportional', pad=0.01, aspect=50)
        if label:
            bar.set_label(label)

    return ax


A set of familiar stars.

In [ ]:
stars = ICRS(['05h14m32.27s', '05h55m10.29s', '05h25m07.87s', '05h36m12.81s', '05h40m45.52s', '05h47m45.39s', '05h32m00.40s',
              '06h45m09.25s', '06h58m37.55s', '07h08m23.49s', '06h22m41.99s', '07h24m05.71s',
              '02h31m47.08s', '14h50m42.40s', '15h20m43.75s', '16h45m58.16s', '15h44m03.46s', '17h32m12.90s', '16h17m30.50s',
              '12h54m01.63s', '11h03m43.84s', '13h47m32.55s', '13h23m55.54s', '11h01m50.39s', '11h53m49.74s', '12h15m25.45s',],
             ['−08d12m05.9s', '+07d24m25.3s', '+06d20m59.0s', '−01d12m06.9s', '−01d56m33.3s', '−09d40m10.6s', '−00d17m56.7s',
              '-16d42m47.3s', '-28d58m19.5s', '−26d23m35.5s', '−17d57m21.3s', '−29d18m11.2s',
              '+89d15m50.9s', '+74d09m19.7s', '+71d50m02.3s', '+82d02m14.1s', '+77d47m40.2s', '+86d35m10.8s', '+75d45m16.9s',
              '+55d57m35.4s', '+61d45m04.0s', '+49d18m47.9s', '+54d55m31.3s', '+56d22m56.4s', '+53d41m41.0s', '+57d01m57.4s'])

Use `init_sky()` to prepare the default [Mollweide](http://usersguidetotheuniverse.com/index.php/2011/03/03/whats-the-best-map-projection/) all-sky projection, which can then be used for other operations like any other axes object:

In [ ]:
ax = init_sky()
p = ax.scatter(ax.projection_ra(stars.ra.degree), ax.projection_dec(stars.dec.degree), marker='.', color='black')

By default, the map is oriented so the DESI footprint does not wrap around in RA and shows the galactic plane in red, but you can change most of this with options:

In [ ]:
ax = init_sky(projection='hammer', ra_center=0, galactic_plane_color=None)
p = ax.scatter(ax.projection_ra(stars.ra.degree), ax.projection_dec(stars.dec.degree), marker='.', color='black')

To see the available projections, use `matplotlib.projections.get_projection_names()`.  The code in this package will probably work best with `'mollweide'`, `'hammer'` or `'aitoff'`, where `'mollweide'` is the default.

In [ ]:
get_projection_names()

## Map Plotting

This section describes two methods that interpret an array of values as a map on the sphere and create a corresponding plot.  If you have a list of (RA, Dec) coordinates that you wish to plot, instead of a map, refer to the next section on catalog plotting.

Use `plot_grid_map()` to map a 2D array of values tabulated on a grid of RA and DEC values, with RA indexed by the first index.  In addition to the grid values, the grid edges must also be specified. Since values are associated with the cells between grid lines, a grid with dimensions `[nRA, nDec]` has corresponding edge arrays of length `nRA+1` and `nDec+1`. The grid must wrap around the sphere in RA but does not need to cover the full range of Dec.  The grid can be chosen independently of the central RA of the map projection and any wrap-around grid column will be split automatically.

In [ ]:
n_ra, n_dec = 15, 10
ra_edges = np.linspace(0., 360., n_ra + 1)
dec_edges = np.linspace(-60., 60., n_dec + 1)
data = np.random.uniform(size=(n_dec, n_ra))
ax = plot_grid_map(data, ra_edges, dec_edges);

Use `plot_map_healpix()` to map a 1D array of values associated with a healpix map. The array size must exactly match the expected size for some value of the healpix `NSIDE` parameter.  Each pixel is rendered as a matplotlib polygon, so this function is somewhat slower than `plot_grid_map()` and also does not automatically wrap around pixels at the RA edges (which are not drawn at all, but this is less visible at larger values of `NSIDE`).

In [ ]:
import healpy as hp

In [ ]:
nside, nest = 4, False
data = np.random.uniform(size=hp.nside2npix(nside))
plot_healpix_map(data, nest);

Use the `colorbar` option to suppress the default colorbar, or the `label` option to add a colorbar label.

## Catalog Plotting

This section describes two methods for plotting a catalog of objects indexed by (RA, DEC).

Use `plot_sky_circles()` to draw a circular field of view centered on each (RA, DEC). This function is useful in DESI for displaying observing tiles and the default field of view is set accordingly.

In [ ]:
plot_sky_circles(ra_center=[0, 60, 120, 180], dec_center=[-30, 0, 30, 60],
                 field_of_view=30);

Note that the projection of a circular field of view is generally not circular. Face colors can be specified for each circle independently.

In [ ]:
plot_sky_circles(ra_center=[0, 60, 120, 180], dec_center=[-30, 0, 30, 60],
                 field_of_view=30, facecolors=['r', 'g', 'b', 'k']);

Each (RA, DEC) can optionally be associated with some value that is used to determine the color of each circle.

In [ ]:
plot_sky_circles(ra_center=[0, 60, 120, 180], dec_center=[-30, 0, 30, 60],
                 field_of_view=30, data=[0, 1, 2, 3], cmap='gray', edgecolor='r');

Use `plot_sky_binned()` to automatically histogram object locations and display the resulting pixel counts per unit sky area as a map, using either the default `plot_grid_map()` or `plot_healpix_map()` (which requires that healpy is installed).

In [ ]:
n = 50000
ra = 360 * np.random.uniform(size=n)
dec = np.clip(20 * np.random.normal(size=n), -90, +90)
plot_sky_binned(ra, dec);

In [ ]:
plot_sky_binned(ra, dec, plot_type='healpix');

The automatic histogramming is controlled by the value (in sq.deg.) of the `max_bin_area` parameter.  The default grid map is binned in cos(DEC) and RA and aims for roughly square pixels on the sky.  Use the `verbose` option to print details of the automatic histogramming. Note that a healpix map is more constrained so will generally result in smaller pixels for the same value of `max_bin_area` (and corrrespondingly larger shot noise).

In [ ]:
plot_sky_binned(ra, dec, max_bin_area=50, verbose=True);

In [ ]:
plot_sky_binned(ra, dec, plot_type='healpix', max_bin_area=50, verbose=True);

Instead of plotting object densities, you can provide an array of data associated with each object and the resulting map will display the mean data value per pixel. Note that empty bins, where the mean data is undefined, are automatically masked.

In [ ]:
plot_sky_binned(ra, dec, data=ra);

## Data Preprocessing

These plotting routines all pass the data to be displayed through `prepare_data()` in order to provide a uniform API for how plot values are masked and mapped to display colors.

In [ ]:
n_ra, n_dec = 15, 10
ra_edges = np.linspace(0., 360., n_ra + 1)
dec_edges = np.linspace(-60., 60., n_dec + 1)
data = np.random.uniform(size=(n_dec, n_ra))
plot_grid_map(data, ra_edges, dec_edges);

For more control, you can call `prepare_data()` yourself before calling the map functions.  For example, you can explicitly mask some pixels in a map:

In [ ]:
data2 = prepare_data(data, mask=(0.4 < data) & (data < 0.6))
plot_grid_map(data2, ra_edges, dec_edges);

You can also specify the clipping to apply at both ends (lo/hi) using a mixture of percentiles or absolute values.

In [ ]:
data2 = prepare_data(data, clip_lo='20%', clip_hi=0.8)
plot_grid_map(data2, ra_edges, dec_edges);

Values outside the clip range are normally displayed (after clipping) but can instead be masked by adding '!' to the clip specification. Note that quotes are optional for numeric clip values (0.8) but required when using '%' or '!'.

In [ ]:
data2 = prepare_data(data, clip_lo='!20%', clip_hi='!0.8')
plot_grid_map(data2, ra_edges, dec_edges);

If the range of clipped data is smaller than the clip limits, the tighter limits will be used for the color scale by default:

In [ ]:
data2 = prepare_data(data, clip_lo=-0.5, clip_hi=1.5)
plot_grid_map(data2, ra_edges, dec_edges);

However, you can force the color scale to use either clip limit by passing `save_limits=True`. This is useful when making a series of plots with a consistent colorscale.

In [ ]:
data2 = prepare_data(data, clip_lo=-0.5, clip_hi=1.5, save_limits=True)
plot_grid_map(data2, ra_edges, dec_edges);

The same techniques can be used with the data passed to `plot_sky_circles()`.

In [ ]:
data = prepare_data([0, 1, 2, 3], mask=[False, True, False, False])
plot_sky_circles(ra_center=[0, 60, 120, 180], dec_center=[-30, 0, 30, 60],
                 field_of_view=30, data=data, edgecolor='k');

The `plot_sky_binned()` function calculates the data to plot internally, but has parameters `clip_lo` and `clip_hi` that are used to prepare this data.

In [ ]:
plot_sky_binned(ra, dec, data=ra, clip_hi='!180');